https://datos.madrid.es/portal/site/egob/menuitem.c05c1f754a33a9fbe4b2e4b284f1a5a0/?vgnextoid=fa8357cec5efa610VgnVCM1000001d4a900aRCRD&vgnextchannel=374512b9ace9f310VgnVCM100000171f5a0aRCRD&vgnextfmt=default

In [13]:
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt

import os
directorios = os.listdir('Data/Climatología')

def read_climate_data(df, cod_magnitud, cod_estacion, magnitud):

    df = df[(df.loc[:, "MAGNITUD"] == cod_magnitud) & (df.loc[:, "ESTACION"] == cod_estacion)]

    cols = ["H01", "H02", "H03", "H04", "H05", "H06", "H07", "H08",
            "H09", "H10", "H11", "H12", "H13", "H14", "H15", "H16",
            "H17", "H18", "H19", "H20", "H21", "H22", "H23", "H24"]

    dic_cols = {}
    for i in range(24):
        dic_cols[cols[i]] = i

    df = df[["ANO", "MES", "DIA"] + cols]

    df = pd.melt(df, id_vars=['ANO', 'MES', 'DIA'], value_vars=cols,
            var_name='HORA', value_name= magnitud)

    df.HORA = df.HORA.map(dic_cols)

    df.loc[:, "Fecha"] = [datetime(df.ANO[i], df.MES[i], df.DIA[i], df.HORA[i]) for i in range(len(df))]
    return df[["Fecha", magnitud]].set_index('Fecha')


temperatura_grouped, humedad_grouped, precipitaciones_grouped, radiacion_grouped, velocidad_grouped = pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame()

for directorio in directorios:
    df = pd.read_csv('Data/Climatología/' + directorio, sep = ";")
#     df.columns = [i.upper() for i in df.columns.tolist()]
#     print(directorio)
    temperaturas = read_climate_data(df, 83, 58, 'Temperatura')
    humedad_relativa = read_climate_data(df, 86, 58, 'Humedad_Relativa')
    precipitaciones = read_climate_data(df, 89, 102, 'Precipitacion')
    radiacion_solar = read_climate_data(df, 88, 102, 'Radiacion')
    velocidad_viento = read_climate_data(df, 81, 102, 'Velocidad_Viento')
    
    temperatura_grouped = pd.concat([temperatura_grouped, temperaturas])
    humedad_grouped = pd.concat([humedad_grouped, humedad_relativa])
    precipitaciones_grouped = pd.concat([precipitaciones_grouped, precipitaciones])
    radiacion_grouped = pd.concat([radiacion_grouped, radiacion_solar])
    velocidad_grouped = pd.concat([velocidad_grouped, velocidad_viento])
    
    temperatura_grouped = temperatura_grouped.sort_index()
    humedad_grouped = humedad_grouped.sort_index()
    precipitaciones_grouped = precipitaciones_grouped.sort_index()
    radiacion_grouped = radiacion_grouped.sort_index()
    velocidad_grouped = velocidad_grouped.sort_index()
    
# Hay atípicos en la temperatura en el 7 y 8 de septiembre de 2021 (todas las horas a -55 grados centígrados)
# Metemos los datos a mano porque son pocos
temperatura_grouped["Temperatura"][temperatura_grouped.loc[:, "Temperatura"] == -55] = [23.6,25.8,28.1,29. ,
            29.9,30.3,31.6,32. ,31.2,29.9,28.5,28. ,27.7,26.2, 23.3,22.2,21.5,20.8,21.1,20.2,19. ,18.2,18.1,19.4,23.6,25.8,28.1,29.]

df_climate =  pd.concat([temperatura_grouped,
                         humedad_grouped,
                         precipitaciones_grouped,
                         radiacion_grouped,
                         velocidad_grouped],
                        axis = 1)

df_climate.loc[:, "ANO"] = df_climate.index.year
df_climate.loc[:, "MES"] = df_climate.index.month
df_climate.loc[:, "DIA"] = df_climate.index.day

In [15]:
temperaturas_maximas = temperatura_grouped.groupby([temperatura_grouped.index.year,
                             temperatura_grouped.index.month,
                             temperatura_grouped.index.day]).max()
temperaturas_minimas = temperatura_grouped.groupby([temperatura_grouped.index.year,
                             temperatura_grouped.index.month,
                             temperatura_grouped.index.day]).min()

spread_temperaturas = temperaturas_maximas - temperaturas_minimas

temperaturas_maximas.index.names = [None, None, None]
temperaturas_maximas.columns = ['Temperatura_max']
temperaturas_minimas.index.names = [None, None, None]
temperaturas_minimas.columns = ['Temperatura_min']
spread_temperaturas.index.names = [None, None, None]
spread_temperaturas.columns = ['Spred_temperatura']

temperaturas_maximas = temperaturas_maximas.reset_index()
temperaturas_minimas = temperaturas_minimas.reset_index()
spread_temperaturas = spread_temperaturas.reset_index()

In [23]:
import yfinance as yf
import datetime 

def read_commodities(ticker, column):
    start = datetime.datetime(2019,1,1) 
    end = datetime.datetime(2022,1,4)
    df = yf.Ticker(ticker)
    df = df.history(start=start, end = end)[["Close"]]
    df = df.asfreq('D', method = 'ffill')
    df.columns = [column]
    df.loc[:, "ANO"] = df.index.year
    df.loc[:, "MES"] = df.index.month
    df.loc[:, "DIA"] = df.index.day
    return df
 
brent = read_commodities("BZ=F", 'Brent')
api2 = read_commodities("MTF=F", 'Api2')
ibex = read_commodities("^IBEX", 'IBEX')

# Reading Commodities

In [25]:
df_climate = df_climate.reset_index().merge(temperaturas_maximas,
                 left_on = ['ANO', 'MES', 'DIA'],
                 right_on = ['level_0', 'level_1', 'level_2']) \
                .merge(temperaturas_minimas,
                 left_on = ['ANO', 'MES', 'DIA'],
                 right_on = ['level_0', 'level_1', 'level_2']) \
                .merge(spread_temperaturas,
                 left_on = ['ANO', 'MES', 'DIA'],
                 right_on = ['level_0', 'level_1', 'level_2']) \
                .merge(brent, on = ['ANO', 'MES', 'DIA']) \
                .merge(api2, on = ['ANO', 'MES', 'DIA']) \
                .merge(ibex, on = ['ANO', 'MES', 'DIA']) \
                .drop(columns = ['level_0_x', 'level_1_x', 'level_2_x',
                                'level_0_y', 'level_1_y','level_2_y',
                                'level_0', 'level_1', 'level_2',
                                'ANO','MES','DIA']) \
                .set_index('Fecha')

In [28]:
df_climate

,Temperatura,Humedad_Relativa,Precipitacion,Radiacion,Velocidad_Viento,Temperatura_max,Temperatura_min,Spred_temperatura,Brent,Api2,IBEX
Fecha,,,,,,,,,,,
2019-01-02 00:00:00,-2.3,91.0,0.0,0.0,0.50,14.1,-5.8,19.9,54.910000,87.449997,8550.000000
2019-01-02 01:00:00,-2.9,91.0,0.0,0.0,0.95,14.1,-5.8,19.9,54.910000,87.449997,8550.000000
2019-01-02 02:00:00,-3.4,91.0,0.0,0.0,0.60,14.1,-5.8,19.9,54.910000,87.449997,8550.000000
2019-01-02 03:00:00,-3.7,92.0,0.0,0.0,0.73,14.1,-5.8,19.9,54.910000,87.449997,8550.000000
2019-01-02 04:00:00,-4.3,91.0,0.0,0.0,1.62,14.1,-5.8,19.9,54.910000,87.449997,8550.000000
...,...,...,...,...,...,...,...,...,...,...,...
2021-12-31 19:00:00,7.2,84.0,0.0,0.0,0.57,17.8,-0.3,18.1,79.199997,136.750000,8713.799805
2021-12-31 20:00:00,5.6,89.0,0.0,0.0,0.12,17.8,-0.3,18.1,79.199997,136.750000,8713.799805
2021-12-31 21:00:00,4.4,91.0,0.0,0.0,0.78,17.8,-0.3,18.1,79.199997,136.750000,8713.799805


In [27]:
df_climate.to_csv('Data/datos_climatologicos.csv')